# Import Modules

In [ ]:
!pip install emojis
!pip install readability
!pip install tqdm
!pip install scikit-learn
!pip install langdetect googletrans
!pip install deep_translator
!pip install language_tool_python
!pip install contractions
!pip install NRCLex

  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.1-py3-none-any.whl size=35461 sha256=5808b5a4d986b88c567ad25223285d9d5b587ac014b636c16ce37a0563f48253
  Stored in directory: /root/.cache/pip/wheels/05/07/4d/2e3a0aaba1713619a403e1a3c56e88a6fc12d753872b98771c
Successfully built readability
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

# Code


In [ ]:
# Importazioni
import sys
import csv
import json
import math
import re
import string
import time
import uuid
from datetime import time as dt_time
import uuid
from langdetect import detect
from googletrans import Translator
import pandas as pd
from nrclex import NRCLex
from html import unescape
import contractions
from textblob import TextBlob
import spacy
from collections import Counter, defaultdict
from itertools import chain
from tqdm import tqdm
import emojis
import language_tool_python
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import readability
from math import log


nltk.download("popular")
stop_words = stopwords.words('english')
csv.field_size_limit(sys.maxsize)


# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/gaiasitri/nltk_data...
[nltk_data]    |   Unzipping corp

In [ ]:
## FUNZIONI

# data nel formato anno-mese.
def extract_date_file(stringa):
    _, _, annomese = stringa.split('.')[0].rpartition('-')
    return annomese

# gestione emoji
def clean_demojize(text):
    demojized_text = emojis.decode(text)
    cleaned_text = re.sub(r':([\w;<>!?/.]+):', r'\1', demojized_text)
    emo_list = emojis.get(text)
    emo_list_decoded = [emojis.decode(el) for el in emo_list]
    emo_types = emojis.count(text, unique=True)
    return cleaned_text, emo_list_decoded, emo_types

# Funzione per parolacce e relativi tag
def leggi_file_tab():
    dizionario = {}
    with open(file_parolacce, 'r') as file_uno:
        for riga in file_uno:
            valori = riga.strip().split('\t')
            dizionario[valori[0]] = valori[1]
    return dizionario

# Funzione per la polarità
def get_sentiment(text):

  blob = TextBlob(text)
  return blob.sentiment.polarity, blob.sentiment.subjectivity

# Funzione per estrarre il conteggio di entità predefinite
def count_named_entities(the_doc, labels):
  entities = defaultdict(int, {key: 0 for key in labels})

  for ent in the_doc.ents:
      if ent.label_ in {"PERSON", "ORG", "GPE", "LOC"}:
          entities[ent.label_] += 1
  return dict(entities)


# Indice di Herdan
def calculate_herdan(list_of_tokens):
  num_tokens = len(list_of_tokens)
  num_types = len(set(list_of_tokens))
  try:
    herdan_index = log(num_types) / log(num_tokens)
  except:
    herdan_index = -1.0
  return round(herdan_index, 2)

# entropia
def calculate_entropy(list_of_tokens):
    token_counter = Counter(list_of_tokens)
    total_tokens = sum(token_counter.values())
    entropy = 0
    for count in token_counter.values():
        probability = count / total_tokens
        if probability > 0:
            entropy -= probability * math.log2(probability)

    return round(entropy, 2)

In [ ]:
file_parolacce = 'lista_parolacce.txt'

In [ ]:
start_time = time.time()


parolacce_dict = leggi_file_tab()
parolacce_labels = list(set(parolacce_dict.values()))
parolacce_data_original = {valore: 0 for valore in parolacce_labels}

out_path = ''
lista_file = ['./File/sample.csv']

lista_sub = ['>', "\(", "<", "\^","~", "\)", "\[", "\]", "\*", "-", "$"]
pos_of_interest = ['NOUN', 'ADV', 'ADJ', 'VERB']
entities_labels = ["PERSON", "ORG", "GPE", "LOC"]
emotion_labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
readability_labels = [\
    'Kincaid', 'ARI', 'Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex',
    'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex', 'characters_per_word',
    'syll_per_word', 'words_per_sentence', 'sentences_per_paragraph', 'type_token_ratio',
    'characters', 'syllables', 'words', 'wordtypes', 'sentences', 'paragraphs',
    'long_words', 'complex_words', 'complex_words_dc'
]


for nome_file in lista_file:


  input_file = out_path + nome_file
  out_file = out_path + nome_file.split('.')[0] + '_emo.csv'

  with open(out_file, 'w') as output_file:
    writer = csv.writer(output_file, delimiter=',')

    with open(input_file,'r') as infile:

      header = [key for key in list(json.loads(infile.readline()).keys()) if key not in ['gildings', 'awardings']]
      header.insert(0, 'id')

      header.extend(chain(
        emotion_labels,
        pos_of_interest,
        parolacce_labels,
        ["PERSON", "ORG", "GPE", "LOC"],
        readability_labels
      ))

      header += ['count_tokens', 'lemmatized_text', 'count_lemmatized_tokens',
                 'upper_words', 'count_upper_words',
                 'lista_emoji', 'count_emoji', 'count_emoji_type',
                 'herdan_value', 'entropy_value', 'count_!', 'count_?', 'count_badwords',
                 'sentiment_polarity_textblob', 'sentiment_subjectivity_textblob']
      writer.writerow(header)

      for line in tqdm(infile):
            # Carica il commento come un dizionario Python
            comment = json.loads(line)
            text = comment['text']

            # Crea Id univoco
            idn = str(uuid.uuid4()) + f'_{extract_date_file(nome_file)}'
            #contrazioni
            text = contractions.fix(text)

            # Decode HTML entities
            text = unescape(text)

            # Remove URLs
            text = re.sub(r'https?://\S+|www\.\S+', '', text)

            # polarità e soggettività
            sentiment_polarity, sentiment_subjectivity = get_sentiment(text)

            #conteggio di "!" e "?"
            conteggio_punti_esclamativi = len(re.findall("!", text))
            conteggio_punti_interrogativi = len(re.findall("\?", text))

            #emoji
            text, lista_emoji, emo_types = clean_demojize(text)

            # Conteggio delle parolacce per categoria
            parolacce_data = defaultdict(int, parolacce_data_original)

            # dizionario per parole piene
            parole_piene = {valore: 0 for valore in pos_of_interest}

            conteggio_parolacce = 0
            token_counts = 0

            doc = nlp(text)
            numero_frasi= len(list(doc.sents))

            lemmatized_tokens, upper_words, pos_tag = [], [], []

            named_entities = count_named_entities(doc, entities_labels)

            total_tokens = len(doc)

            for token in doc:
              if not token.is_stop:
                token_counts += 1
                categoria = parolacce_dict.get(token.text.lower())

                if categoria is not None:
                  conteggio_parolacce += 1
                  parolacce_data[categoria] = parolacce_data.get(categoria, 0) + 1


                if token.is_upper and len(token) > 1:
                  upper_words.append(token.text)

                if not token.is_punct and not any(char.isdigit() for char in token.text):
                  lemmatized_tokens.append(token.lemma_.lower())
                  pos_tag.append(token.pos_)

                  if parole_piene.get(token.pos_, None) != None:
                    parole_piene[token.pos_] += 1

            # Herdan
            herdan_value = calculate_herdan(lemmatized_tokens)

            # Entropy
            entropy_value = calculate_entropy(lemmatized_tokens)

            lemmatized_text = " ".join(lemmatized_tokens)


            for el in lista_sub:
              lemmatized_text = re.sub(el, ' ', lemmatized_text)
            # Rimozione menzione utenti (@username)
            lemmatized_text = re.sub(r'@\w+', ' ', lemmatized_text)
            # Remove new lines and tabs
            lemmatized_text = re.sub(r'[\n\t]', ' ', lemmatized_text)
            # al posto del \ metti lo spazio
            lemmatized_text = re.sub(r'\/', ' ', lemmatized_text)
            # Riduce spazi multipli
            lemmatized_text = re.sub(r'\s+', ' ', lemmatized_text).strip()

            # Emozioni
            affect_frequencies_dict = NRCLex(lemmatized_text).raw_emotion_scores

            # Estrazione readability grades e sentence info
            try:
              r = readability.getmeasures(lemmatized_text, lang='en')
              readability_data = dict(r['readability grades'].items())| dict(r['sentence info'].items())
            except:
              readability_data = None

            # Output
            output_list = [idn]
            output_list += [comment[key] for key in comment.keys() if key not in ['gildings', 'awardings']]

            emotion_out_list = [affect_frequencies_dict.get(key, None) for key in emotion_labels]
            pos_of_interest_counters = [parole_piene.get(key, None) for key in pos_of_interest]
            bad_words_out_list = [parolacce_data[key] for key in parolacce_labels]
            ner_out_list = [named_entities[key] for key in entities_labels]
            if readability_data:
              readability_data_values = [readability_data.get(key, None) for key in readability_labels]
            else:
              readability_data_values = [None] * len(readability_labels)

            output_list.extend(chain(emotion_out_list, pos_of_interest_counters, bad_words_out_list, ner_out_list, readability_data_values))

            output_list.extend([total_tokens, lemmatized_text, len(lemmatized_tokens),
                                  '_'.join(upper_words), len(upper_words),
                                  '_'.join(lista_emoji), len(lista_emoji) , emo_types,
                                  herdan_value, entropy_value,
                                  conteggio_punti_esclamativi, conteggio_punti_interrogativi, conteggio_parolacce,
                                  sentiment_polarity, sentiment_subjectivity])

            writer.writerow(output_list)

end_time = time.time()

# Calcola il tempo trascorso
elapsed_time = end_time - start_time

# Converti il tempo in minuti
elapsed_minutes = elapsed_time / 60

print(f"\nTempo di esecuzione: {elapsed_minutes:.2f} minuti")

49it [00:01, 38.45it/s]


Tempo di esecuzione: 0.03 minuti
